# Проект "чат-бот"

В чат-боте реализована оценка запроса, в результате которого идет либо обращение в "болталку" (ответы с мэйл.ру) либо поиск по действительно поисковому запросу - данные id товара и его title

## Импортируем библиотеки

In [1]:
import os
import string
import annoy

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

## Предобработка текста, чтобы обучить word2vec и получить эмбеддинги. Удаляем знаки препинания и делаем лемматизацию

In [184]:
# Функция предобработки текста
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

## Работа над "болталкой"

In [5]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

<ipython-input-5-22e9d148421c>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [185]:
# Обучаем модель word2vec на вопросах sentences
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
model.save("w2v_model")

In [2]:
question = None
written = False

#Мы идем по всем записям, берем первую строку как вопрос
# и после знака --- находим ответ
with open("prepared_answers.txt", "w") as fout:
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<ipython-input-2-2f773e5bf4ec>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

Теперь нам нужно сложить в индекс все вопросы. Используем библиотеку annoy. Проходимся по всем ответам, считаем, что вектор предложения - усредненная сумма word2vecов слов, которые входят в него

In [7]:
# Индексируем
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1

index.build(10)
index.save('speaker.ann')

<ipython-input-7-461dd520c7f8>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


0it [00:00, ?it/s]

True

## Работа над продуктовым запросом

In [186]:
# Загружаем датасет, убирая лишние столбцы
modul_dataset = pd.read_csv('ProductsDataset.csv')
cols = ['product_id', 'title']
modul_dataset = modul_dataset[cols]

# Убираем строки с пустыми product_id
modul_dataset = modul_dataset.dropna(subset=['product_id'])

# Реализуем столбец для будущих ответов
modul_dataset['answer'] = modul_dataset[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

modul_dataset.head()

,product_id,title,answer
0,58e3cfe6132ca50e053f5f82,Юбка детская ORBY,58e3cfe6132ca50e053f5f82 Юбка детская ORBY
1,5667531b2b7f8d127d838c34,Ботильоны,5667531b2b7f8d127d838c34 Ботильоны
2,59534826aaab284cba337e06,Брюки,59534826aaab284cba337e06 Брюки
3,57de544096ad842e26de8027,Продам детские шапки,57de544096ad842e26de8027 Продам детские шапки
4,5ad4d2626c86cb168d212022,Блузка,5ad4d2626c86cb168d212022 Блузка


In [187]:
# Обучаем модель word2vec на названиях товаров title

titles = []
for line in  modul_dataset['title']:
    title = preprocess_txt(line)
    titles.append(title)
    c += 1
    
titles_for_model = [i for i in titles if len(i)>2]

model_product = Word2Vec(sentences=titles_for_model, vector_size=100, min_count=1, window=5)
model_product.save("w2v_product_model")

In [188]:
# Индексируем
index_product = annoy.AnnoyIndex(100 ,'angular')

index_map_product = {}
counter_product = 0

for line in modul_dataset['answer'].str.split(' ', n=1):
    n_w2v_product = 0
    index_map_product[counter_product] = line[0] + ' ' + line[1]
    question = preprocess_txt(line[1])
    vector_product = np.zeros(100)
    for word in question:
        if word in model_product.wv:
            vector_product += model_product.wv[word]
            n_w2v_product += 1
    if n_w2v_product > 0:
        vector_product = vector_product / n_w2v_product
    index_product.add_item(counter_product, vector_product)
            
    counter_product += 1

index_product.build(10)
index_product.save('speaker_product.ann')

True

## Обучаем классификатор запроса - продуктовый или болтовня (Логистическая регрессия)

In [189]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Собираем датасет для построения модели

In [244]:
# Датасет из болталки
df_sentences = pd.DataFrame()
df_sentences['question'] = sentences
df_sentences['question'] = df_sentences['question'].apply(' '.join)
df_sentences['target'] = 0

In [245]:
# Датасет из продуктовых данных
df_titles = pd.DataFrame()
df_titles['question'] = titles_for_model
df_titles['question'] = df_titles['question'].apply(' '.join)
df_titles['target'] = 1

In [246]:
# Итоговый датасет
df_result = pd.concat([df_sentences, df_titles])

In [247]:
# Разбиваем на train - test
x = df_result.question
y = df_result.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [248]:
# Векторизируем
vectorizer = TfidfVectorizer()
values = vectorizer.fit(x_train, y_train)

In [249]:
x_train = values.transform(x_train)
y_train = np.array(y_train)

x_test = values.transform(x_test)
y_test = np.array(y_test)

In [250]:
# Логистическая регрессия
clf_model = LogisticRegression()
clf_model.fit(x_train, y_train)

LogisticRegression()

In [251]:
# Предсказываем значения на тестовой выборке
predictions = clf_model.predict(x_test)

# Точность классификации
print(accuracy_score(y_test, predictions))

0.9898188529836417


## Сводим обе модели (болтовня + запрос) с классификатором и заворачиваем в одну функцию get_answer()

In [253]:
# Итоговая функция по модели
def get_answer(question):
    """Функция получения ответа на вопрос"""
    preprocessed_question = preprocess_txt(question)
    preprocessed_question = " ".join(c for c in preprocessed_question)
    n_w2v = 0
    vector = np.zeros(100)
    question_vec = vectorizer.transform([preprocessed_question])
    print(clf_model.predict(question_vec))
    if clf_model.predict(question_vec) == 0:  # Определяем болтовня (0) или запрос (1)
        for word in preprocessed_question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        answer_index = index.get_nns_by_vector(vector, 1)
        return index_map[answer_index[0]]  # Выводим ответ
    elif clf_model.predict(question_vec) == 1:   # Определяем болтовня (0) или запрос (1)
        for word in preprocessed_question:
            if word in model_product.wv:
                vector += model_product.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        answer_index = index_product.get_nns_by_vector(vector, 1)
        return index_map_product[answer_index[0]] # Выводим ответ

In [220]:
# Тестовая функция для болтовни
# def get_answer_0(question):
#     preprocessed_question = preprocess_txt(question)
#     n_w2v = 0
#     vector = np.zeros(100)
#     for word in preprocessed_question:
#         if word in model.wv:
#             vector += model.wv[word]
#             n_w2v += 1
#     if n_w2v > 0:
#         vector = vector / n_w2v
#     answer_index = index.get_nns_by_vector(vector, 1)
#     return index_map[answer_index[0]]

In [221]:
# Тестовая функция для продуктового запроса
# def get_answer_1(question):
#     preprocessed_question = preprocess_txt(question)
#     n_w2v = 0
#     vector = np.zeros(100)
#     for word in preprocessed_question:
#         if word in model_product.wv:
#             vector += model_product.wv[word]
#             n_w2v += 1
#     if n_w2v > 0:
#         vector = vector / n_w2v
#     answer_index = index_product.get_nns_by_vector(vector, 1)
#     return index_map_product[answer_index[0]]

In [264]:
get_answer('брюки')

[1]


'59afc7a0dbdf0f353234bb03 Летние шорты, размер 48-50'